<a href="https://colab.research.google.com/github/ninyx/dl_zero_shot/blob/main/ZeroShot_KWS_ImageBind.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing Dependencies


In [2]:
!pip install torch
!pip install torchvision
!pip install torchaudio
!pip install pytorchvideo @ git+https://github.com/facebookresearch/pytorchvideo.git@28fe037d212663c6a24f373b94cc5d478c8c1a1d
!pip install timm==0.6.7
!pip install ftfy
!pip install regex
!pip install einops
!pip install fvcore
!pip install decord==0.6.0
!pip install iopath
!pip install numpy
!pip install matplotlib
!pip install types-regex
!pip install mayavi
!pip install cartopy
!pip install soundfile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Invalid requirement: '@'
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.0/510.0 kB 11.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.o

## Installing ImageBind by MetaAI

In [3]:
!git clone https://github.com/facebookresearch/ImageBind.git


Cloning into 'ImageBind'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 89 (delta 28), reused 20 (delta 17), pack-reused 48
Unpacking objects: 100% (89/89), 2.64 MiB | 10.18 MiB/s, done.
--2023-06-04 12:34:35--  https://dl.fbaipublicfiles.com/imagebind/imagebind_huge.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 99.84.160.46, 99.84.160.108, 99.84.160.61, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|99.84.160.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4803584173 (4.5G) [binary/octet-stream]
Saving to: ‘imagebind_huge.pth’

imagebind_huge.pth  100%[===================>]   4.47G   225MB/s    in 24s     

2023-06-04 12:34:59 (194 MB/s) - ‘imagebind_huge.pth’ saved [4803584173/4803584173]



## Testing Image Bind

In [8]:
!pip install pytorchvideo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 21.2 MB/s eta 0:00:00
  Created wheel for pytorchvideo: filename=pytorchvideo-0.1.5-py3-none-any.whl size=188694 sha256=dbd1a9648c207b4b45a867dbeaa6657d83192d6c4127a94b99fec1727520d290
  Stored in directory: /root/.cache/pip/wheels/ff/4e/81/0f72a543be9ed7eb737c95bfc5da4025e73226b44368074ece
Successfully built pytorchvideo


In [7]:
%cd /content/ImageBind

/content/ImageBind


In [8]:
import data
import torch
from models import imagebind_model
from imagebind_model import ModalityType

In [9]:
text_list=["A dog.", "A car", "A bird"]
image_paths=[".assets/dog_image.jpg", ".assets/car_image.jpg", ".assets/bird_image.jpg"]
audio_paths=[".assets/dog_audio.wav", ".assets/car_audio.wav", ".assets/bird_audio.wav"]

device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Instantiate model
model = imagebind_model.imagebind_huge(pretrained=True)
model.eval()
model.to(device)

# Load data
inputs = {
    ModalityType.TEXT: data.load_and_transform_text(text_list, device),
    ModalityType.VISION: data.load_and_transform_vision_data(image_paths, device),
    ModalityType.AUDIO: data.load_and_transform_audio_data(audio_paths, device),
}

with torch.no_grad():
    embeddings = model(inputs)

print(
    "Vision x Text: ",
    torch.softmax(embeddings[ModalityType.VISION] @ embeddings[ModalityType.TEXT].T, dim=-1),
)
print(
    "Audio x Text: ",
    torch.softmax(embeddings[ModalityType.AUDIO] @ embeddings[ModalityType.TEXT].T, dim=-1),
)
print(
    "Vision x Audio: ",
    torch.softmax(embeddings[ModalityType.VISION] @ embeddings[ModalityType.AUDIO].T, dim=-1),
)

100%|██████████| 4.47G/4.47G [00:58<00:00, 82.6MB/s]


Vision x Text:  tensor([[9.9684e-01, 3.1310e-03, 2.5929e-05],
        [5.4494e-05, 9.9993e-01, 2.0353e-05],
        [4.4847e-05, 1.3246e-02, 9.8671e-01]], device='cuda:0')
Audio x Text:  tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]], device='cuda:0')
Vision x Audio:  tensor([[8.2460e-01, 8.0997e-02, 9.4405e-02],
        [1.4594e-01, 6.6145e-01, 1.9261e-01],
        [1.1730e-03, 7.4875e-04, 9.9808e-01]], device='cuda:0')


## Import KWS dataset
